In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_train.csv')

In [ ]:
# um metodo generico de arrumas as variáveis 
housing = df.copy()

# comecando com o preenchimento dos espacos vazios
for column in housing.columns:
    if housing[column].dtype != 'object':
        housing[column].fillna(housing[column].mean(), inplace= True)
    else:
        housing[column] = housing[column].astype('str')
        housing[column].fillna(housing[column].mode(), inplace= True)

In [ ]:
# conferindo as estatísticas das variáveis
for column in housing.columns:
    if housing[column].dtype == 'object':
        print(column, '\n', housing[column].value_counts(), '\n')
    else:
        print(column, '\n', housing[column].describe(), '\n')

In [ ]:
# ajustando um primeiro modelo de regressão 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score

# separando em variáveis dependentes e independentes
x = housing.drop(['SalePrice'], axis= 1)
y = housing['SalePrice']

# aplicando a codificacao das labels
le = LabelEncoder()
for column in x.columns:
    if x[column].dtype == 'object':
        x[column] = le.fit_transform(x[column])

# separando treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size= 0.2, random_state= 55)

In [ ]:
# ajustando cross validation
clf1_cv = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

cross_val = cross_val_score(clf1_cv, x, y, cv= 5)

In [ ]:
# chamando o modelo
clf1 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

# ajustando um modelo
clf1.fit(x_treino, y_treino)

# fazendo a previsao
prev = clf1.predict(x_teste)

# avaliando o erro
from sklearn.metrics import mean_squared_log_error
import numpy as np

erro = np.sqrt(mean_squared_log_error(y_teste, prev))
erro

In [ ]:
# fazendo a previsao nos dados de teste
# lendo os dados
url_data = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_test.csv'
data = pd.read_csv(url_data)

In [ ]:
# ocupando as lacunas do dataframe com o mesmo metodo que o treino
for column in data.columns:
    if data[column].dtype != 'object':
        data[column].fillna(data[column].mean(), inplace= True)
    else:
        data[column] = data[column].astype('str')
        data[column].fillna(data[column].mode(), inplace= True)

# transformando dos dados com o label encoder 
le = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column])

In [ ]:
# dataframe a ser enviado
final = clf1.predict(data)
envio = pd.DataFrame({'Id': data['Id'], 'SalePrice': final})

In [ ]:
# salvando o df para envio
path = r'C:\Users\Avell\Desktop\Python\github\datasets'
nome = '\housing_submission_1.csv'

envio.to_csv(path+nome, index= False)